In [1]:
!pip install pyspark

**Now Load the File Using PySpark**

In [2]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("ManufacturingMonitor").getOrCreate()

# Read the uploaded CSV file
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Source Data/dataset.csv", header=True, inferSchema=True)

# Display the schema
df.printSchema()

# Show sample data
df.show(5)


root
 |-- ID: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- SensorReading: double (nullable = true)
 |-- MachineStatus: string (nullable = true)
 |-- QualityCheck: boolean (nullable = true)
 |-- ProductCount: integer (nullable = true)

+---+--------------------+------------------+-------------+------------+------------+
| ID|           Timestamp|     SensorReading|MachineStatus|QualityCheck|ProductCount|
+---+--------------------+------------------+-------------+------------+------------+
|  1|2023-11-22 09:24:...|43.081004305053725|      Standby|        true|          21|
|  2|2023-11-21 09:24:...| 71.31249095658937|           On|        true|          18|
|  3|2023-11-20 09:24:...|19.096744555580937|      Standby|       false|          10|
|  4|2023-11-19 09:24:...|28.951475117230828|           On|        true|          24|
|  5|2023-11-18 09:24:...|46.283913967493504|      Standby|        true|          23|
+---+--------------------+------------------+--

**Clean and Transform the Manufacturing Data**

In [4]:
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.types import FloatType, DoubleType, IntegerType, LongType

# Get data types of each column
numeric_types = (FloatType, DoubleType, IntegerType, LongType)

null_exprs = []

for column in df.columns:
    data_type = [field.dataType for field in df.schema.fields if field.name == column][0]
    if isinstance(data_type, numeric_types):
        null_exprs.append(count(when(col(column).isNull() | isnan(col(column)), column)).alias(column))
    else:
        null_exprs.append(count(when(col(column).isNull(), column)).alias(column))

df.select(null_exprs).show()


+---+---------+-------------+-------------+------------+------------+
| ID|Timestamp|SensorReading|MachineStatus|QualityCheck|ProductCount|
+---+---------+-------------+-------------+------------+------------+
|  0|        0|            0|            0|           0|           0|
+---+---------+-------------+-------------+------------+------------+



**Export Cleaned Data from PySpark to CSV (for Tableau)**

In [5]:
# Save to CSV (in Google Colab environment)
df.coalesce(1).write.option("header", "true").csv("/content/cleaned_manufacturing_data")

In [6]:
# Zip the output folder so you can download it
!zip -r /content/cleaned_manufacturing_data.zip /content/cleaned_manufacturing_data

  adding: content/cleaned_manufacturing_data/ (stored 0%)
  adding: content/cleaned_manufacturing_data/part-00000-1d7d013d-9e7c-4c93-b139-742583de3c87-c000.csv (deflated 70%)
  adding: content/cleaned_manufacturing_data/.part-00000-1d7d013d-9e7c-4c93-b139-742583de3c87-c000.csv.crc (stored 0%)
  adding: content/cleaned_manufacturing_data/_SUCCESS (stored 0%)
  adding: content/cleaned_manufacturing_data/._SUCCESS.crc (stored 0%)


In [7]:
from google.colab import files
files.download("/content/cleaned_manufacturing_data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Project Summary: Automated Production Defect Dashboard using Python & Tableau**

### **Objective:**

To build an automated and interactive Tableau dashboard that visualizes production defects and helps identify performance issues across different time periods, product lines, and shifts using data from a dynamic Excel file.

---

###  **Data Source:**

* **Excel file** containing:

  * Date of Inspection
  * Product Line
  * Shift (A/B/C)
  * Total Units Inspected
  * Total Defective Units

---

### **Key Features & Metrics:**

1. **Defect Rate Calculation:**

   * Formula: `(Defects / Total Inspected) × 100`
   * Created as a **calculated field** in Tableau.

2. **Interactive Charts Developed:**

   * **Defect Trend Over Time** (Line Chart)

     * Shows how defect rate changes daily or monthly.
   * **Defect Rate by Shift** (Bar Chart)

     * Compares defect rates across Shift A, B, and C.
   * **Defect Rate by Product Line** (Bar or Tree Map)

     * Highlights which product lines have higher defects.

3. **Filters Added:**

   * Date Range Selector
   * Shift Filter
   * Product Line Filter

4. **Tooltips and Labels:**

   * Dynamic percentages shown in all visualizations.
   * Enhanced user interaction with visual tooltips.

---

### **Automation Setup:**

* Tableau is connected to the Excel file such that any update to the Excel data:

  * Automatically reflects in Tableau upon data refresh.
  * No manual re-import or rework required.

---

### **Business Impact:**

* Enables production teams and quality engineers to:

  * Monitor real-time defect trends.
  * Identify problematic shifts or product lines.
  * Make quick decisions for process improvements.
